In [41]:
import pandas as pd
df = pd.read_csv('/content/sample_data/03_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [42]:
%pip install catboost

In [43]:
%pip install lightbgm

ERROR: Could not find a version that satisfies the requirement lightbgm (from versions: none)
ERROR: No matching distribution found for lightbgm


In [56]:
from lightgbm import LGBMRegressor
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost
from sklearn.metrics import make_scorer, mean_squared_error

candidate = Catboost(random_state=42)

param_distribs = {
    'iterations': randint(low=6, high=80),
    'depth': randint(low=5, high=25),
}

# Define the scoring function
scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

param_search = RandomizedSearchCV(
    candidate,
    param_distributions=param_distribs,
    n_iter=15,
    random_state=42,
    scoring=scorer,
    cv=3,
    n_jobs=-1
    )

param_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
15 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", l

Learning rate set to 0.5
0:	learn: 1.0767496	total: 162ms	remaining: 8.93s
1:	learn: 1.0688557	total: 307ms	remaining: 8.29s
2:	learn: 1.0635605	total: 466ms	remaining: 8.24s
3:	learn: 1.0624238	total: 615ms	remaining: 8s
4:	learn: 1.0617398	total: 779ms	remaining: 7.94s
5:	learn: 1.0601147	total: 932ms	remaining: 7.77s
6:	learn: 1.0597942	total: 1.1s	remaining: 7.7s
7:	learn: 1.0596299	total: 1.25s	remaining: 7.48s
8:	learn: 1.0594827	total: 1.41s	remaining: 7.34s
9:	learn: 1.0593767	total: 1.56s	remaining: 7.16s
10:	learn: 1.0592403	total: 1.7s	remaining: 6.95s
11:	learn: 1.0591408	total: 1.86s	remaining: 6.84s
12:	learn: 1.0588869	total: 2.01s	remaining: 6.66s
13:	learn: 1.0588181	total: 2.16s	remaining: 6.49s
14:	learn: 1.0587560	total: 2.32s	remaining: 6.35s
15:	learn: 1.0585329	total: 2.57s	remaining: 6.43s
16:	learn: 1.0584037	total: 2.88s	remaining: 6.61s
17:	learn: 1.0583375	total: 3.2s	remaining: 6.75s
18:	learn: 1.0582716	total: 3.49s	remaining: 6.79s
19:	learn: 1.0579771	to

RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostRegressor object at 0x7fde0e34e9e0>,
                   n_iter=15, n_jobs=-1,
                   param_distributions={'depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fde0df92ad0>,
                                        'iterations': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fde0e34fd90>},
                   random_state=42,
                   scoring=make_scorer(mean_squared_error, greater_is_better=False, response_method='predict', squared=False))

In [57]:
param_search.best_params_

{'depth': 7, 'iterations': 56}

In [58]:
best_model = param_search.best_estimator_

In [59]:
from sklearn.metrics import root_mean_squared_error
y_pred = best_model.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0549267156016329

In [60]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [61]:
y_pred = best_model.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.0593807283638133

In [62]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/content/sample_data/03_test.csv')
del test_set['Premium Amount']
y_pred = best_model.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('Stacking_Hyper_Tuning_03.csv', index=False)